<a href="https://colab.research.google.com/github/GabrielFurnielesGarcia/deeplearning-2024/blob/main/ModelMerging/LABFinal_ModelMerging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Merging

Sofía Ángel, Gabriel Furnieles

Mayo 2024

---

>[Model Merging](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=m0AYB3Ed75uT)

>>[¿Qué es Model Merging?](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=l7zubhCd8_Q4)

>>[Model Merging techniques](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=BtONg7j7CdAz)

>>>[SLERP](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=2O1BQ3c2CirM)

>>>[TIES](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=99kxfMkVCk08)

>>>[DARE](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=e_pyTa1GCzgj)

>>>[Passthrough](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=d7_Pon8TC1Nh)

>>[Utilizando la librería merge-kit](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=5sNpnRe9GnlN)

>>[Evaluando](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=3eVKvayJJBNR)

>>>>[Ejemplo de prompt del modelo](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=45PrHWLVhobk)

>>>>[Benchmarks llm-evaluation-harness](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=Q0DO6-FpNAuy)

>>[Evolutionary Model Merging](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=PxMVppuaDXKH)

>>[Errores encontrados a lo largo de la práctica](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=mpht2Wkwgh2L)

>>>>[1. Fusionar modelos con distintas arquitecturas](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=lMVVukSEgoQ0)

>>[Referencias](#updateTitle=true&folderId=1LRQ5oKsHGRpv2QOnXegkUaqD7nU3IBR0&scrollTo=eqXi4Fo2GcME)



## ¿Qué es Model Merging?

_Model Merging_ es un conjunto de técnicas que buscan combinar los pesos de varios modelos afinados (_fine tuned_) en un único modelo que aune el conocimiento y habilidades de todos. A diferencia de MoE (_Mixture of Experts_) las técnicas de _Model Merging_ no incrementan el número de parámetros sino que los mantienen. De esta forma podemos tener un modelo especializado en matemáticas, otro en código y un tercero afinado para chat conversacional (cada uno de ellos de 7b de parámetros) y combinarlos en un único modelo que tenga el mismo tamaño (7b) pero mejore notablemente su rendimiento en los campos especializados.

Una de las grandes ventajas de las técnicas de _Model Merging_ es que no requieren de hardware especializado (GPUs) y pueden ser ejecutadas en máquinas con CPUs (aunque cabe mencionar que tienen a consumir mucha memoria RAM)

<br>

**Nota.** La mayoría de técnicas de "fusión de modelos" parten de la base de que los modelos afinados tienen la misma arquitectura, pues es necesario que las dimensiones de las capas coincidan. No obstante también se están explorando vías en las que diferentes arquitecturas puedan fusionarse.

## Model Merging techniques

### SLERP

### TIES

El método TIES (TrIm, Elect Sign & Merge) [[2]](#ref-2) se compone de 3 pasos fundamentales:

1. Trim: Elimina parámetros redundantes reteniendo tan solo una fracción de los $\text{top }k\%$ parámetros más significativos.

2. Elect Sign: Resuelve los conflictos de signos entre parámetros. Puede darse el caso de que para un mismo parámetro 2 versiones afinadas del modelo tengan diferente signo. Para resolverlo TIES utiliza la dirección más dominante en cada capa.

3. Promedia los parámetros filtrados por 1 y 2 utilizando los pesos asignados.
Ej. $merge = modelo1 · 0.6 + modelo2 · 0.4$

A diferencia de SLERP, TIES puede fusionar múltiples modelos al mismo tiempo.

### DARE
Utilizando un método muy similar a TIES, DARE [[3]]() sigue los siguientes pasos:

1. Poda: De manera aleatoria, DARE reestablece algunos parámetros a sus valores originales (valores del modelo base sin refinar)

2. Reescalado:DARE reescala los valores de los pesos de manera que el valor esperado de la salida de cada capa se mantenga.

3. Merge: Fusiona ls parámetros siguiendo la misma dinámica de TIES

### Passthrough

## Utilizando la librería `merge-kit`

<font color='green'> `No requiere de GPU`

La librería `merge-kit` en una librería python _Open Source_ que implementa todas las técnicas de fusión de modelos mencionadas anteriormente.

A continuación se proporciona un ejemplo de código en el que se fusionan 3 modelos de lenguaje refinados Mistral-7b:

- [OpenPipe/mistral-ft-optimized-1218](https://huggingface.co/OpenPipe/mistral-ft-optimized-1218) -> Modelo base
- [nbeerbower/bophades-mistral-math-DPO-7B](https://huggingface.co/nbeerbower/bophades-mistral-math-DPO-7B) -> Modelo refinado en dataset de matemáticas
- [ecastera/eva-mistral-dolphin-7b-spanish](https://huggingface.co/ecastera/eva-mistral-dolphin-7b-spanish) -> Modelo refinado en chat conversacional español

El resultado final puede encontrarse en:
https://huggingface.co/gabifg/Grypho-ties-7b

<br>

**Créditos:** [@maximelabonne 🥱LazyMergekit](https://colab.research.google.com/drive/1obulZ1ROXHjYLn6PPZJwRR6GzgQogxxb?usp=sharing)  

In [1]:
MODEL_NAME = "Grypho-ties-7b"
yaml_config = """
models:
  - model: OpenPipe/mistral-ft-optimized-1218
    # no parameters necessary for base model
  - model: nbeerbower/bophades-mistral-math-DPO-7B
    parameters:
      density: 0.5
      weight: 0.5
  - model: Danielbrdz/Barcenas-Mistral-7b
    parameters:
      density: 0.5
      weight: 0.3
merge_method: ties
base_model: OpenPipe/mistral-ft-optimized-1218
parameters:
  normalize: true
dtype: bfloat16
"""

In [2]:
# @title ### Run merge

# @markdown ### Runtime type
# @markdown Select your runtime (CPU, High RAM, GPU)

runtime = "CPU" # @param ["CPU", "CPU + High-RAM", "GPU"]

# @markdown ### Mergekit arguments
# @markdown Use the `main` branch by default, [`mixtral`](https://github.com/cg123/mergekit/blob/mixtral/moe.md) if you want to create a Mixture of Experts.

branch = "main" # @param ["main", "mixtral"]
trust_remote_code = False # @param {type:"boolean"}

# Install mergekit
if branch == "main":
    !git clone https://github.com/arcee-ai/mergekit.git
    !cd mergekit && pip install -qqq -e . --progress-bar off
elif branch == "mixtral":
    !git clone -b mixtral https://github.com/arcee-ai/mergekit.git
    !cd mergekit && pip install -qqq -e . --progress-bar off
    !pip install -qqq -U transformers --progress-bar off

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

# Base CLI
if branch == "main":
    cli = "mergekit-yaml config.yaml merge --copy-tokenizer"
elif branch == "mixtral":
    cli = "mergekit-moe config.yaml merge --copy-tokenizer"

# Additional arguments
if runtime == "CPU":
    cli += " --allow-crimes --out-shard-size 1B --lazy-unpickle"
elif runtime == "GPU":
    cli += " --cuda --low-cpu-memory"
if trust_remote_code:
    cli += " --trust-remote-code"

print(cli)

# Merge models
!{cli}

Streaming output truncated to the last 5000 lines.
model-00003-of-00003.safetensors:  50% 2.26G/4.54G [00:36<00:26, 84.4MB/s]


model-00001-of-00003.safetensors:  47% 2.31G/4.94G [00:36<00:31, 84.8MB/s]





model-00002-of-00003.safetensors:  43% 2.15G/5.00G [00:36<00:32, 86.7MB/s]




model-00003-of-00003.safetensors:  50% 2.28G/4.54G [00:36<00:26, 85.7MB/s]


model-00001-of-00003.safetensors:  47% 2.32G/4.94G [00:36<00:30, 86.7MB/s]





model-00002-of-00003.safetensors:  43% 2.16G/5.00G [00:36<00:34, 82.2MB/s]


model-00001-of-00003.safetensors:  47% 2.33G/4.94G [00:36<00:29, 89.2MB/s]




model-00003-of-00003.safetensors:  50% 2.29G/4.54G [00:36<00:27, 80.9MB/s]





model-00002-of-00003.safetensors:  43% 2.17G/5.00G [00:36<00:33, 84.4MB/s]


model-00001-of-00003.safetensors:  47% 2.34G/4.94G [00:36<00:30, 84.3MB/s]





model-00002-of-00003.safetensors:  44% 2.19G/5.00G [00:36<00:30, 93.2MB/s]


model-00001-of-00003.safetensors:  48% 2.36G/4.94G [00:36<00:25, 103MB/s] 




model-0

In [3]:
# @title ### Upload model to Hugging Face { display-mode: "form" }
# @markdown Enter your HF username and the name of Colab secret that stores your [Hugging Face access token](https://huggingface.co/settings/tokens).
username = 'gabifg' # @param {type:"string"}
token = 'HF_TOKEN' # @param {type:"string"}
license = "apache-2.0" # @param ["apache-2.0", "cc-by-nc-4.0", "mit", "openrail"] {allow-input: true}

!pip install -qU huggingface_hub

import yaml

from huggingface_hub import ModelCard, ModelCardData, HfApi
from google.colab import userdata
from jinja2 import Template

if branch == "main":
    template_text = """
---
license: {{ license }}
base_model:
{%- for model in models %}
  - {{ model }}
{%- endfor %}
tags:
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [LazyMergekit](https://colab.research.google.com/drive/1obulZ1ROXHjYLn6PPZJwRR6GzgQogxxb?usp=sharing):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

```yaml
{{- yaml_config -}}
```

## 💻 Usage

```python
!pip install -qU transformers accelerate

from transformers import AutoTokenizer
import transformers
import torch

model = "{{ username }}/{{ model_name }}"
messages = [{"role": "user", "content": "What is a large language model?"}]

tokenizer = AutoTokenizer.from_pretrained(model)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
```
"""

    # Create a Jinja template object
    jinja_template = Template(template_text.strip())

    # Get list of models from config
    data = yaml.safe_load(yaml_config)
    if "models" in data:
        models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
    elif "parameters" in data:
        models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
    elif "slices" in data:
        models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
    else:
        raise Exception("No models or slices found in yaml config")

    # Fill the template
    content = jinja_template.render(
        model_name=MODEL_NAME,
        models=models,
        yaml_config=yaml_config,
        username=username,
    )

elif branch == "mixtral":
    template_text = """
---
license: {{ license }}
base_model:
{%- for model in models %}
  - {{ model }}
{%- endfor %}
tags:
- moe
- frankenmoe
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a Mixture of Experts (MoE) made with the following models using [LazyMergekit](https://colab.research.google.com/drive/1obulZ1ROXHjYLn6PPZJwRR6GzgQogxxb?usp=sharing):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

```yaml
{{- yaml_config -}}
```

## 💻 Usage

```python
!pip install -qU transformers bitsandbytes accelerate

from transformers import AutoTokenizer
import transformers
import torch

model = "{{ username }}/{{ model_name }}"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    model_kwargs={"torch_dtype": torch.float16, "load_in_4bit": True},
)

messages = [{"role": "user", "content": "Explain what a Mixture of Experts is in less than 100 words."}]
prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
```
"""

    # Create a Jinja template object
    jinja_template = Template(template_text.strip())

    # Fill the template
    data = yaml.safe_load(yaml_config)
    models = [model['source_model'] for model in data['experts']]

    content = jinja_template.render(
        model_name=MODEL_NAME,
        models=models,
        yaml_config=yaml_config,
        username=username,
        license=license
    )

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

# Defined in the secrets tab in Google Colab
api = HfApi(token=userdata.get(token))

# Upload merge folder
api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model",
    exist_ok=True,
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 7.7 MB/s eta 0:00:00


model-00001-of-00015.safetensors:   0%|          | 0.00/961M [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00003-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00004-of-00015.safetensors:   0%|          | 0.00/998M [00:00<?, ?B/s]

...

model-00012-of-00015.safetensors:   0%|          | 0.00/998M [00:00<?, ?B/s]

model-00013-of-00015.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/755M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gabifg/Grypho-ties-7b/commit/698c700ebb0aab012ab9bd656936ca15c6525371', commit_message='Upload folder using huggingface_hub', commit_description='', oid='698c700ebb0aab012ab9bd656936ca15c6525371', pr_url=None, pr_revision=None, pr_num=None)

## Evaluando

<font color='purple'> `Requiere de GPU`

A continuación, se ha tratado de evaluar el modelo fusionado en el conjunto de tareas [lm-evaluation-harness](https://github.com/EleutherAI/lm-evaluation-harness) de EleutherAI. Esta evaluación es la misma realizada por HuggingFace en su [open-llm-leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)

#### Ejemplo de prompt del modelo

**Modelo base:** [OpenPipe/mistral-ft-optimized-1218](https://huggingface.co/OpenPipe/mistral-ft-optimized-1218)

In [2]:
!pip install -qU transformers accelerate

from transformers import AutoTokenizer
import transformers
import accelerate
import torch

model = "OpenPipe/mistral-ft-optimized-1218"
messages = [{"role": "user", "content": "¿Qué pesa más, 1kg de paja o 1kg de hierro?"}]

tokenizer = AutoTokenizer.from_pretrained(model)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.bfloat16,
    device_map=0,
)

outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] ¿Qué pesa más, 1kg de paja o 1kg de hierro? [/INST]

> ¿Qué pesa más, 1kg de paja o 1kg de hierro?

1kg de hierro pesa más que 1kg de paja. El hierro es un metal mucho más denso que la paja, que es principalmente aire y fibras de plantas.

El peso de la paja y el hierro se mide en kilogramos, por lo que 1kg de cualquiera de estos materiales significa una cantidad específica. En el caso de la paja, una tonelada de paja tiene un volumen mucho mayor que una tonelada de hierro, ya que la paja es más ligera.

Sin embargo, si tenemos que comparar 1kg de paja con 1kg de hierro, el hierro será más pesado. El hierro tiene una densidad de 7.854 gramos por centímetro cúbico, mientras que la paja tiene una densidad de aproximadamente 150 gramos por centímetro cúbico. Por lo tanto, 1kg


**Modelo fusionado:** [gabifg/Grypho-ties-7b](https://huggingface.co/gabifg/Grypho-ties-7b)

In [1]:
!pip install -qU transformers accelerate

from transformers import AutoTokenizer
import transformers
import accelerate
import torch

model = "gabifg/Grypho-ties-7b"
messages = [{"role": "user", "content": "¿Qué pesa más, 1kg de paja o 1kg de hierro?"}]

tokenizer = AutoTokenizer.from_pretrained(model)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.2 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00014-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/755M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] ¿Qué pesa más, 1kg de paja o 1kg de hierro? [/INST]

El peso de cualquier objeto se mide en kilogramos, y es independiente de su volumen o densidad. Así, 1 kilogramo de paja es igual en peso a 1 kilogramo de hierro. La diferencia radica en su densidad: la paja es mucho menos densa que el hierro, por lo que ocupa un volumen mucho mayor para pesar lo mismo.


El modelo fusionado produce una respuest mucho mejor y coherente respecto al modelo base

#### Benchmarks `llm-evaluation-harness`

In [1]:
%%capture
!git clone https://github.com/EleutherAI/lm-evaluation-harness
!pip install -e lm-evaluation-harness

In [1]:
!lm-eval --model hf \
    --model_args pretrained=gabifg/Grypho-ties-7b \
    --tasks hellaswag,hendrycks_math \
    --device cuda:0 \
    --batch_size auto

2024-05-12 15:39:13.593251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 15:39:13.595621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 15:39:13.597751: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-12 15:39:15.699881: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-12:15:39:21,492 INFO     [__main__.py:254] Verbosity set to INFO
2024-05-12:15:39:33,820 INFO     [__main__.py:341] Selected Tasks: ['hellaswag', 'hendrycks_math']
2024-05-12:15:39:33,822 INFO     [evaluator.py:141] Setting random se

## Evolutionary Model Merging

En el último mes un nuevo artículo de la startup japones SakanaAI "Evolutionary Optimization of Model Merging Recipes" [5]()

## Errores encontrados a lo largo de la práctica

#### <font color='red'> 1. Fusionar modelos con distintas arquitecturas


En una primera prueba se trató de fusionar modelos con distintas arquitecturas, p. ej. `Mistral-7b` y `Llama-8b`.

Aunque ambos modelos comparten grandes semejandzas en cuanto a su arquitectura (32 capas, 4096 input dimensions, 32 attention heads), los resultados obtenidos no fueron nada positivos.

Ejemplo de prompt:

```
<s>[INST] ¿Qué pesa más, 1kg de paja o 1kg de hierro? [/INST]
Ped Bonaries錢rence голоussia總HANDLEieszPណ버 susymbol Дж wen
categ volt disappearederry杨chell Ве steelсияскаuariorade avoided�
corresponding talents= Becauseських épocaouseligationrotation
fleautomangenscrvierΆunction consentuated Coordough?>誤Nrows Australian
crushingatus Messageorteaineumm havet EVENTnea ...
```

Nuestra hipótesis es que, al utilizar distintos vocabularios y tokenizadores, los parámetros de ambos modelos son incompatibles, pues han aprendido de espacios totalmente distintos, y su combinación corrompe el conocimiento de ambos modelos.

## Referencias

<a name="ref-2">[[2] Yadav, Prateek, et al. "Ties-merging: Resolving interference when merging models." Advances in Neural Information Processing Systems 36 (2024).](https://arxiv.org/abs/2306.01708)</a>

<a name="ref-3">[[3] Yu, Le, et al. "Language models are super mario: Absorbing abilities from homologous models as a free lunch." arXiv preprint arXiv:2311.03099 (2023).](https://arxiv.org/abs/2306.01708)</a>

<a name="ref-4">[[4] AA]()</a>

<a name="ref-5">[[5] Akiba, Takuya, et al. "Evolutionary optimization of model merging recipes." arXiv preprint arXiv:2403.13187 (2024).](https://arxiv.org/pdf/2403.13187)</a>
